Extraction of thermal data from single image

In [1]:
from flirimageextractor import FlirImageExtractor

# Initialize the extractor
flir = FlirImageExtractor()

# Load the FLIR image
flir.process_image("/home/deepaksr/project/project_assignment1/data4/FLIR/Test/car/flir_20200116T174847.jpg")

# Extract raw thermal data (in Celsius)
thermal_image = flir.get_thermal_np()

# Print the thermal data
print(thermal_image)

[[5.49892365 5.50872554 5.57730786 ... 2.20166771 2.20166771 2.20166771]
 [5.49892365 5.50872554 5.5675137  ... 2.21185489 2.21185489 2.21185489]
 [5.49892365 5.50872554 5.55771844 ... 2.28313071 2.29330805 2.29330805]
 ...
 [3.55593382 3.55593382 3.55593382 ... 3.58600748 3.58600748 3.58600748]
 [3.54590691 3.54590691 3.55593382 ... 3.58600748 3.58600748 3.58600748]
 [3.54590691 3.54590691 3.55593382 ... 3.58600748 3.58600748 3.58600748]]


Conversion of camera images to single channel dataset

In [2]:
import os
import cv2
from flirimageextractor import FlirImageExtractor
import numpy as np

# Initialize the extractor
flir = FlirImageExtractor()

# Paths to your dataset and output directories
input_dir = "/home/deepaksr/project/project_assignment1/data4/FLIR"
output_dir = "/home/deepaksr/project/project_assignment1/thermal_output"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def process_and_save_thermal_images(input_dir, output_dir):
    # Traverse through the dataset
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".jpg"):  # Only process .jpg files
                img_path = os.path.join(root, file)
                
                # Extract thermal data from the image
                try:
                    flir.process_image(img_path)
                    thermal_data = flir.get_thermal_np()

                    # Normalize the thermal data for saving as image (if needed)
                    # Optionally, you can apply normalization or scaling
                    thermal_data_normalized = cv2.normalize(thermal_data, None, 0, 255, cv2.NORM_MINMAX)
                    thermal_image = thermal_data_normalized.astype(np.uint8)

                    # Prepare the corresponding output directory
                    relative_path = os.path.relpath(root, input_dir)
                    output_subdir = os.path.join(output_dir, relative_path)
                    os.makedirs(output_subdir, exist_ok=True)

                    # Save the thermal image as a grayscale image (single channel)
                    output_img_path = os.path.join(output_subdir, file)
                    cv2.imwrite(output_img_path, thermal_image)

                    print(f"Processed and saved: {output_img_path}")
                except Exception as e:
                    print(f"Failed to process {img_path}: {e}")

# Run the processing function
process_and_save_thermal_images(input_dir, output_dir)


Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200307T204747.jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200116T174919.jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200116T174928 (2).jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200116T175019 (2).jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200307T210724.jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200307T210732.jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200307T204429 (2).jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_output/Train/car/flir_20200307T204635.jpg
Processed and saved: /home/deepaksr/project/project_assignment1/thermal_outp

Conversion to 4 channel dataset

In [2]:
import os
import cv2
from flirimageextractor import FlirImageExtractor
import numpy as np

# Initialize the extractor
flir = FlirImageExtractor()

# Paths to your dataset and output directories
input_dir = "/home/deepaksr/project/project_assignment1/data4/FLIR"
output_dir = "/home/deepaksr/project/project_assignment1/thermal_rgba_png_output"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def process_and_save_thermal_rgb_images(input_dir, output_dir):
    # Traverse through the dataset
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".jpg"):  # Only process .jpg files
                img_path = os.path.join(root, file)
                
                # Extract thermal data from the image
                try:
                    # Process the FLIR image to extract thermal data
                    flir.process_image(img_path)
                    thermal_data = flir.get_thermal_np()

                    # Normalize the thermal data for consistency
                    thermal_data_normalized = cv2.normalize(thermal_data, None, 0, 255, cv2.NORM_MINMAX)
                    thermal_image = thermal_data_normalized.astype(np.uint8)  # Convert to uint8 (grayscale)

                    # Load the corresponding RGB image
                    rgb_image = cv2.imread(img_path)
                    if rgb_image is None:
                        print(f"Failed to load RGB image at {img_path}")
                        continue

                    # Resize the RGB image to match the thermal image dimensions
                    rgb_image_resized = cv2.resize(rgb_image, (thermal_image.shape[1], thermal_image.shape[0]))

                    # Add the thermal image as the 4th channel to the resized RGB image
                    thermal_image_expanded = np.expand_dims(thermal_image, axis=2)  # Make it 3D (H, W, 1)
                    rgba_image = np.concatenate((rgb_image_resized, thermal_image_expanded), axis=2)  # Combine as (H, W, 4)

                    # Prepare the corresponding output directory
                    relative_path = os.path.relpath(root, input_dir)
                    output_subdir = os.path.join(output_dir, relative_path)
                    os.makedirs(output_subdir, exist_ok=True)

                    # Save the RGBA image as PNG
                    output_img_path = os.path.join(output_subdir, file.replace(".jpg", ".png"))  # Save as PNG
                    cv2.imwrite(output_img_path, rgba_image)

                    print(f"Processed and saved RGBA image: {output_img_path}")
                except Exception as e:
                    print(f"Failed to process {img_path}: {e}")

# Run the processing function
process_and_save_thermal_rgb_images(input_dir, output_dir)


Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200307T204747.png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200116T174919.png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200116T174928 (2).png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200116T175019 (2).png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200307T210724.png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200307T210732.png
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_png_output/Train/car/flir_20200307T204429 (2).png
Processed and saved RGBA image: /home/deepaksr/pro

In [1]:
import os
import cv2
from flirimageextractor import FlirImageExtractor
import numpy as np

# Initialize the extractor
flir = FlirImageExtractor()

# Paths to your dataset and output directories
input_dir = "/home/deepaksr/project/project_assignment1/data4/FLIR"
output_dir = "/home/deepaksr/project/project_assignment1/thermal_rgba_output"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def process_and_save_thermal_rgb_images(input_dir, output_dir):
    # Traverse through the dataset
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".jpg"):  # Only process .jpg files
                img_path = os.path.join(root, file)
                
                # Extract thermal data from the image
                try:
                    # Process the FLIR image to extract thermal data
                    flir.process_image(img_path)
                    thermal_data = flir.get_thermal_np()

                    # Normalize the thermal data for consistency
                    thermal_data_normalized = cv2.normalize(thermal_data, None, 0, 255, cv2.NORM_MINMAX)
                    thermal_image = thermal_data_normalized.astype(np.uint8)  # Convert to uint8 (grayscale)

                    # Load the corresponding RGB image
                    rgb_image = cv2.imread(img_path)
                    if rgb_image is None:
                        print(f"Failed to load RGB image at {img_path}")
                        continue

                    # Resize the RGB image to match the thermal image dimensions
                    rgb_image_resized = cv2.resize(rgb_image, (thermal_image.shape[1], thermal_image.shape[0]))

                    # Add the thermal image as the 4th channel to the resized RGB image
                    thermal_image_expanded = np.expand_dims(thermal_image, axis=2)  # Make it 3D (H, W, 1)
                    rgba_image = np.concatenate((rgb_image_resized, thermal_image_expanded), axis=2)  # Combine as (H, W, 4)

                    # Prepare the corresponding output directory
                    relative_path = os.path.relpath(root, input_dir)
                    output_subdir = os.path.join(output_dir, relative_path)
                    os.makedirs(output_subdir, exist_ok=True)

                    # Save the RGBA image
                    output_img_path = os.path.join(output_subdir, file)
                    cv2.imwrite(output_img_path, rgba_image)

                    print(f"Processed and saved RGBA image: {output_img_path}")
                except Exception as e:
                    print(f"Failed to process {img_path}: {e}")

# Run the processing function
process_and_save_thermal_rgb_images(input_dir, output_dir)


Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200307T204747.jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200116T174919.jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200116T174928 (2).jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200116T175019 (2).jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200307T210724.jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200307T210732.jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/thermal_rgba_output/Train/car/flir_20200307T204429 (2).jpg
Processed and saved RGBA image: /home/deepaksr/project/project_assignment1/the